In [6]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer

In [71]:
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [ ]:
viz = MeshcatVisualizer(robot)

In [74]:
IDX_LF = cmodel.getFrameId('leg_left_1_link')
cpin.framesForwardKinematics(cmodel, cdata, robot.q0)

ArgumentError: Python argument types in
    pinocchio.pinocchio_pywrap_casadi.framesForwardKinematics(Model, Data, numpy.ndarray)
did not match C++ signature:
    framesForwardKinematics(pinocchio::ModelTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> model, pinocchio::DataTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} data, Eigen::MatrixBase<Eigen::Matrix<casadi::Matrix<casadi::SXElem>, -1, 1, 0, -1, 1> > q)

In [68]:
for i,f in enumerate(cmodel.frames):
    print(i, f.name)

0 universe
1 root_joint
2 base_link
3 leg_left_1_joint
4 leg_left_1_link
5 leg_left_2_joint
6 leg_left_2_link
7 leg_left_3_joint
8 leg_left_3_link
9 leg_left_4_joint
10 leg_left_4_link
11 leg_left_5_joint
12 leg_left_5_link
13 leg_left_6_joint
14 leg_left_6_link
15 leg_left_sole_fix_joint
16 left_sole_link
17 leg_right_1_joint
18 leg_right_1_link
19 leg_right_2_joint
20 leg_right_2_link
21 leg_right_3_joint
22 leg_right_3_link
23 leg_right_4_joint
24 leg_right_4_link
25 leg_right_5_joint
26 leg_right_5_link
27 leg_right_6_joint
28 leg_right_6_link
29 leg_right_sole_fix_joint
30 right_sole_link
31 torso_1_joint
32 torso_1_link
33 torso_2_joint
34 torso_2_link
35 arm_left_1_joint
36 arm_left_1_link
37 arm_left_2_joint
38 arm_left_2_link
39 arm_left_3_joint
40 arm_left_3_link
41 arm_left_4_joint
42 arm_left_4_link
43 arm_left_5_joint
44 arm_left_5_link
45 arm_left_6_joint
46 arm_left_6_link
47 arm_left_7_joint
48 arm_left_7_link
49 wrist_left_ft_joint
50 wrist_left_ft_link
51 wrist_left_t

In [72]:
viz.display(robot.q0)

In [75]:
class KineModel:
    def __init__ (self, cmodel):
        self.cmodel = cmodel
        self.cdata = cdata = cmodel.createData()
        self.nq, self.nv = cmodel.nq, cmodel.nv
        self.nx = self.nq+ self.nv

        cq = casadi.SX.sym('cq', self.nq, 1)
        cv = casadi.SX.sym('cv', self.nv, 1)
        ca = casadi.SX.sym('ca', self.nv, 1)
        cx = casadi.vertcat(cq, cv)

        IDX_LF = cmodel.getFrameId('leg_left_1_link')

        self.com_position = casadi.Function('com', [cq], [cpin.centerOfMass(cmodel, cdata, cq)] )

        cpin.framesForwardKinematics(cmodel, cdata, cq, cv, ca)
        cpin.updateFramePlacement(cmodel, cdata)

        self.lf_position = casadi.Function('lf', [cq], cdata.oMf[IDX_LF].translation)
    
    def cost(self, cq):
        com_pos = self.com_position(cq)
        return com_pos.T @ com_pos

    #def constraints(self, cq, opti):
        

In [76]:
kine_model = KineModel(cmodel)

opti = casadi.Opti()

dxs = opti.variable(kine_model.nx, 1)

opti.minimize(kine_model.cost(dxs[: kine_model.nq]))

ArgumentError: Python argument types in
    pinocchio.pinocchio_pywrap_casadi.framesForwardKinematics(Model, Data, SX, SX, SX)
did not match C++ signature:
    framesForwardKinematics(pinocchio::ModelTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> model, pinocchio::DataTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} data, Eigen::MatrixBase<Eigen::Matrix<casadi::Matrix<casadi::SXElem>, -1, 1, 0, -1, 1> > q)

In [46]:
opti.solver('ipopt')

opti.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      777

Total number of variables............................:       77
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

OptiSol(Opti {
  instance #7
  #variables: 1 (nx = 77)
  #parameters: 0 (np = 0)
  #constraints: 0 (ng = 0)
  CasADi solver allocated.
  CasADi solver was called: Solve_Succeeded
})

In [48]:
x_sol = opti.value(dxs)
q_sol = x_sol[: kine_model.nq]
v_sol = x_sol[kine_model.nq:]

In [49]:
viz.display(q_sol)